<a href="https://colab.research.google.com/github/rubanzasilva/playgroundSeries-S03E01/blob/testA/ps_e01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
from fastai.tabular.all import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/Rubanza/datasets/playground-series-s3e1/

sample_submission.csv  test.csv  train.csv


In [ ]:
path = Path('drive/MyDrive/Rubanza/datasets/playground-series-s3e1/')
path

Path('drive/MyDrive/Rubanza/datasets/playground-series-s3e1')

In [ ]:
train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
sample_df = pd.read_csv(path/'sample_submission.csv')
train_df


,id,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980
1,1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946
2,2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576
3,3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336
4,4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500
...,...,...,...,...,...,...,...,...,...,...
37132,37132,3.3438,50.0,4.936508,1.079365,1775.0,3.022222,34.19,-118.36,1.856
37133,37133,3.7308,26.0,5.087533,0.966019,1006.0,4.316901,37.32,-121.86,1.588
37134,37134,4.1716,52.0,4.678862,1.101485,1156.0,1.431734,37.75,-122.44,3.387
37135,37135,2.7143,16.0,5.710074,1.068376,584.0,2.803659,38.40,-120.98,1.592


In [ ]:
train_df

,id,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980
1,1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946
2,2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576
3,3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336
4,4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500
...,...,...,...,...,...,...,...,...,...,...
37132,37132,3.3438,50.0,4.936508,1.079365,1775.0,3.022222,34.19,-118.36,1.856
37133,37133,3.7308,26.0,5.087533,0.966019,1006.0,4.316901,37.32,-121.86,1.588
37134,37134,4.1716,52.0,4.678862,1.101485,1156.0,1.431734,37.75,-122.44,3.387
37135,37135,2.7143,16.0,5.710074,1.068376,584.0,2.803659,38.40,-120.98,1.592


Define fastai dataloader

In [ ]:
dls = TabularDataLoaders.from_csv(path/'train.csv', path=path, y_names="MedHouseVal",
    cat_names = [],
    cont_names = ['id','MedInc'	,'HouseAge'	,'AveRooms'	,'AveBedrms',	'Population',	'AveOccup',	'Latitude',	'Longitude'],
    procs = [Categorify, FillMissing, Normalize])

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

In [ ]:
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = [],
    cont_names = ['id','MedInc'	,'HouseAge'	,'AveRooms'	,'AveBedrms',	'Population',	'AveOccup',	'Latitude',	'Longitude'],
                   y_names='MedHouseVal',
                   splits=splits)

In [ ]:
to.xs.iloc[:2]

,id,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
34190,1.456587,-0.355986,1.807124,0.516691,1.523573,-0.88309,-0.076480,-0.737992,0.536445
25110,0.608953,0.158456,-0.412071,0.318277,0.079679,-0.04172,-0.116131,-0.915743,0.779674


In [ ]:
dls = to.dataloaders(bs=64)

In [ ]:
learn = tabular_learner(dls, metrics=rmse)

In [ ]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,_rmse,time
0,3.154050,2.418456,1.555139,00:07
1,0.541056,0.462419,0.680014,00:04
2,0.480764,0.570753,0.755482,00:04
3,0.467777,0.453022,0.673069,00:03
4,0.449125,0.403650,0.635335,00:03
5,0.440294,0.499073,0.706451,00:04
6,0.429264,0.394876,0.628391,00:03
7,0.442252,0.391329,0.625563,00:03
8,0.403969,0.388614,0.623389,00:03
9,0.406777,0.384929,0.620427,00:03


predictions on test set

In [ ]:
dl = learn.dls.test_dl(test_df)

In [ ]:
preds = learn.get_preds(dl=dl)
preds

(tensor([[0.6871],
         [1.3550],
         [3.6938],
         ...,
         [1.0619],
         [3.5464],
         [3.4092]]), None)

Upload kaggle API and submit

In [ ]:
# Next, install the Kaggle API client.
!pip install -q kaggle
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rubanzasilva","key":"069494dac9599dfa6ad2faeffa757e38"}'}

In [ ]:

 ! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

# List available datasets.
#!kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
target_preds = preds[0]

In [ ]:
test_df['MedHouseVal'] = target_preds

test_df.to_csv('submission.csv', columns=['id','MedHouseVal'], index=False)

sub = pd.read_csv('submission.csv')
sub.head()

,id,MedHouseVal
0,37137,0.687073
1,37138,1.354979
2,37139,3.693830
3,37140,3.755415
4,37141,2.587302


In [ ]:
!kaggle competitions submit -c playground-series-s3e1 -f submission.csv -m "08th sub 1"

100% 383k/383k [00:04<00:00, 82.8kB/s]
Successfully submitted to Playground Series Season 3, Episode 1